# Homework II

Gonçalo Bárias (ist1103124) & Raquel Braunschweig (ist1102624)

This Notebook only contains our solution for the programming and critical analysis section of the Homework (Part II). Our solution for the pen and paper section (Part I) can be found on the pdf report.

## II. Programming and critical analysis [7v]

**Considering the `column_diagnosis.arff` dataset available at the course webpage’s homework tab. Using `sklearn`, apply a 10-fold stratified cross-validation with shuffling (`random_state=0`) for the
assessment of predictive models along this section.**

### 1) [3v]

**Compare the performance of kNN with k = 5 and naïve Bayes with Gaussian assumption
(consider all remaining parameters for each classifier as `sklearn`'s default):**

- **(a) Plot two boxplots with the fold accuracies for each classifier.**

- **(b) Using `scipy`, test the hypothesis "kNN is statistically superior to naïve Bayes regarding accuracy", asserting whether is true.**

### 2) [2.5v]

**Consider two kNN predictors with k = 1 and k = 5 (uniform weights, Euclidean distance,
all remaining parameters as default). Plot the differences between the two cumulative confusion
matrices of the predictors. Comment.**

### 3) [1.5v]

**Considering the unique properties of `column_diagnosis`, identify three possible difficulties
of naïve Bayes when learning from the given dataset.**